In [ ]:
%reload_ext autoreload
%autoreload 2
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
import cv2 as cv
import os
import rasterio
import numpy as np
import PIL.Image
from rasterio.windows import Window
from kidney.datasets.kaggle import outlier, get_reader, DatasetReader, SampleType
from kidney.inference.window import sliding_window_boxes
from kidney.utils.tiff import read_tiff

In [ ]:
reader = get_reader()

In [ ]:
test_keys = reader.get_keys(SampleType.Unlabeled)

In [ ]:
dirname = "/tmp/boxes"
os.makedirs(dirname, exist_ok=True)
for key in test_keys:
    meta = reader.fetch_meta(key)
    with rasterio.open(meta["tiff"]) as dataset:
        h, w = dataset.shape
        boxes = sliding_window_boxes(w, h, 512, 32)
        for i, box in enumerate(boxes, 1):
            x1, y1, x2, y2 = box
            crop = dataset.read([1, 2, 3], window=Window.from_slices((y1, y2), (x1, x2)))
            filename = os.path.join(dirname, f"{key}_{x1}_{y1}_{x2}_{y2}.png")
            PIL.Image.fromarray(np.moveaxis(crop, 0, -1)).save(filename)
            print(f"Image saved: {i}")
    break

In [ ]:
import glob
sizes = {PIL.Image.open(filename).size for filename in glob.glob(f"{dirname}/*.png")}

In [ ]:
sizes

In [ ]:
img = read_tiff(meta["tiff"])

In [ ]:
img = cv.UMat(img)

In [ ]:
for box in boxes:
    x1, y1, x2, y2 = box
    cv.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    break

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img.get())